In [44]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
import os

In [41]:
# set paths
# if don't have Tessract binary, get it here and install for one user: 
# https://github.com/UB-Mannheim/tesseract/wiki
CAPTCHAPNG = os.path.join(os.path.expanduser(r'~\Downloads'),'CIKRFcaptcha.png')
pytesseract.pytesseract.tesseract_cmd = os.path.expanduser(r'~\AppData\Local\Tesseract-OCR\tesseract.exe')

In [6]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - There is no [win32] chromedriver for browser 85.0.4183 in cache
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/85.0.4183.87/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\ai\.wdm\drivers\chromedriver\win32\85.0.4183.87]


In [55]:
def CIKRFget(url,driver,curAttempt=0,maxAttempt=20):
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content,"html.parser")
    captchaFlag = soup.select('#captchaImg')
    if captchaFlag:
        with open(CAPTCHAPNG, 'wb') as file:
            file.write(driver.find_element_by_xpath('//*[@id="captchaImg"]').screenshot_as_png)
        img = Image.open(CAPTCHAPNG)
        

        text = pytesseract.image_to_string(img, config='digits').strip()
        formfield = driver.find_elements_by_xpath('//*[@id="captcha"]')[0]
        formfield.send_keys(text)
        button = driver.find_elements_by_xpath('//*[@id="send"]')[0]
        button.click()
        time.sleep(5)
        if BeautifulSoup(driver.page_source,"html.parser").select('#captchaImg') and curAttempt< maxAttempt:
            CIKRFget(url,driver,curAttempt+1,maxAttempt)
        elif BeautifulSoup(driver.page_source,"html.parser").select('#captchaImg') and curAttempt>= maxAttempt:
            return None
        else:
            return driver.page_source

In [62]:
testURL = 'http://www.vybory.izbirkom.ru/region/izbirkom?action=show&vrn=4024029414046&region=3&prver=0&pronetvd=0'
test = CIKRFget(testURL,driver,curAttempt=0,maxAttempt=20)

In [63]:
test

'<html><head>\n<base href="http://www.bashkortostan.vybory.izbirkom.ru/">\n<link href="/css/report.css" rel="stylesheet" type="text/css">\n<title>Сведения о проводящихся выборах и референдумах</title>\n<meta http-equiv="pragma" content="no-cache">\n<meta http-equiv="cache-control" content="no-cache">\n<meta http-equiv="expires" content="0">\n\n<script type="text/javascript" async="" src="http://www.google-analytics.com/ga.js"></script><script type="text/javascript">\n\t\n\t\tfunction open_close(group_id)\n\t\t{\t\t\t\n\t\t\tfolder = document.getElementById(group_id + "1");\n\t\t\ti = 1;\n\t\t\twhile (folder != null) {\n\t\t\t\tif (folder.className == \'trFinrepHidden\') {\n\t\t\t\t\tfolder.className = \'trFinrep\';\n\t\t\t\t} else {\n\t\t\t\t\tfolder.className = \'trFinrepHidden\';\n\t\t\t\t}\n\t\t\t\ti++;\t\t\n\t\t\t\tfolder = document.getElementById(group_id + "" + i);\t\t\n\t\t\t}\t\t\n\t\t}\n\t\n</script>\n\n</head>\n\n<body text="#000000" link="#0000ff" vlink="#0000ff" alink="#ff0